In [86]:
import geopandas as gpd
import pandas as pd
import os
import parcelfunks
import numpy as np
from scipy import stats
import shutil
from glob import glob
import importlib
import numpy as np

In [103]:
CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
#CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
CO_pInventory = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\ColoradoInventory.csv', dtype={'STATE':str,'COUNTY':str})
CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv')) # need to get this to work
fipsList = CO_pInventory_True['COUNTY'].tolist()   
parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]

In [105]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
unionDF = pd.DataFrame()
joinDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    if os.path.exists(os.path.join(path,fips+'double20.gpkg')):
        union = gpd.read_file(os.path.join(path, fips+'double20.gpkg'),layer='MH_parc_blk_union')
        unionDF = pd.concat([unionDF,union])
        join = gpd.read_file(os.path.join(path, fips+'double20.gpkg'),layer='MH_parcels')
        joinDF = pd.concat([joinDF,join])        
unionDF.to_file(os.path.join(co_22_dir,'Colorado_Final_double20.gpkg'),layer='Colorado_Final_union')
#joinDF['polyLen_3'] = joinDF.apply(lambda row: geomLen(row.geometry), axis=1)
#joinDF['geomZscore_3'] = np.abs(stats.zscore(joinDF['polyLen2']))
joinDF.to_file(os.path.join(co_22_dir, 'Colorado_Final_double20.gpkg'),layer='Colorado_Final_MH_parcels')

In [9]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
final_near_join = gpd.read_file(os.path.join(co_22_dir, 'Colorado_Final_multi100_less5.gpkg'), layer='Colorado_Final_MH_parcels')
final_near_join.to_csv(os.path.join(co_22_dir, 'Colorado_Final_multi100_less5_parcels.csv'))

Blow up old ones

In [4]:
for fips, path in zip(fipsList,parcelsPaths):
    if os.path.exists(os.path.join(path,fips+'multi100_less5.gpkg')):
        os.remove((os.path.join(path,fips+'multi100_less5.gpkg')))

Getting non-matched mhps:

In [8]:
mhp_path = os.path.join(co_22_dir,'08_MHPS_OG.gpkg')
mhps = gpd.read_file(mhp_path,layer='08_MHPs_Prepped_OG')
MHparcel_path = os.path.join(co_22_dir,'Colorado_Final_multi100_less5.gpkg')
MHparcels = gpd.read_file(MHparcel_path, layer='Colorado_Final_MH_parcels')
mhps_merge = pd.merge(mhps,MHparcels, on = 'MHPID', how='outer')
mhps_nonmatched = mhps_merge.loc[mhps_merge['geometry_y']==None].copy()
mhps_nonmatched.drop(mhps_nonmatched.filter(regex='_y$').columns, axis=1, inplace=True)
renames_x = mhps_nonmatched.filter(regex='_x$').columns
renames = [x.split('_x')[0] for x in renames_x]
renames = dict(zip(renames_x,renames))
mhps_nonmatched.rename(renames, axis='columns',inplace=True)
drop_cols = mhps_nonmatched.columns[11:]
mhps_nonmatched.drop(drop_cols,axis=1,inplace=True)
mhps_nonmatched.to_file(mhp_path,layer='08_mhps_non_matched_multi100_less5')

Building Area stuff:

In [4]:
#OG building aggregator:
def sumWithin(df1,df2):
    dfsjoin = gpd.sjoin(df1,df2,predicate='contains')
    dfpivot = pd.pivot_table(dfsjoin,index=['ID','APN'], aggfunc={'FID':len}).reset_index()
    dfpolynew = df1.merge(dfpivot, how='left', on='ID')
    dfpolynew.rename({'APN_x':'APN', 'FID': 'Sum_Within'}, axis='columns',inplace=True)
    dfpolynew['Sum_Within'].fillna(0, inplace=True)
    dfpolynew.drop(['ID','APN_y'], axis=1, inplace=True)
    return dfpolynew

In [12]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
county_dir = os.path.join(co_22_dir,'counties')
adams_dir = os.path.join(county_dir,'08001')

In [41]:
adams = gpd.read_file(os.path.join(adams_dir,'parcels.shp')).to_crs(crs='ESRI:102003')
adamsBuildings = gpd.read_file(os.path.join(adams_dir,'08001_buildings.shp')).to_crs(crs='ESRI:102003')

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08001\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [42]:
adams['ID'] = adams.index
adamsBuildings['areas'] = adamsBuildings['geometry'].area

In [43]:
dfsjoin = gpd.sjoin(adams,adamsBuildings,predicate='contains')

In [16]:
dfsjoin.columns

Index(['APN', 'APN2', 'STATE', 'COUNTY', 'FIPS', 'SIT_HSE_NU', 'SIT_DIR',
       'SIT_STR_NA', 'SIT_STR_SF', 'SIT_FULL_S', 'SIT_CITY', 'SIT_STATE',
       'SIT_ZIP', 'SIT_ZIP4', 'LAND_VALUE', 'IMPR_VALUE', 'TOT_VALUE',
       'ASSMT_YEAR', 'REC_DATE', 'SALES_PRIC', 'YEAR_BUILT', 'STD_LAND_U',
       'LOT_SIZE', 'BLDG_AREA', 'DEED_DSCR', 'NO_OF_STOR', 'NO_OF_UNIT',
       'BEDROOMS', 'BATHROOMS', 'OWNER', 'OWNER2', 'OWNADDRESS', 'OWNADDRES2',
       'OWNCTYSTZP', 'XCOORD', 'YCOORD', 'MINX', 'MINY', 'MAXX', 'MAXY',
       'VERSION', 'QUANTARIUM', 'ATTDATE', 'geomCalc', 'geometry',
       'index_right', 'FID', 'County', 'areas'],
      dtype='object')

In [20]:
dfsjoin.iloc[0:2]

,APN,APN2,STATE,COUNTY,FIPS,SIT_HSE_NU,SIT_DIR,SIT_STR_NA,SIT_STR_SF,SIT_FULL_S,...,MAXY,VERSION,QUANTARIUM,ATTDATE,geomCalc,geometry,index_right,FID,County,areas
4,155700000006,5,CO,Adams,08001,16704,None,MIMOSA MILE,RD,16704 MIMOSA MILE RD,...,40.000949,12/20/2016,22098024,20211030,187514.4901,"POLYGON ((-680680.353 308693.941, -681061.940 ...",2826,39041,Adams County,218.498917
4,155700000006,5,CO,Adams,08001,16704,None,MIMOSA MILE,RD,16704 MIMOSA MILE RD,...,40.000949,12/20/2016,22098024,20211030,187514.4901,"POLYGON ((-680680.353 308693.941, -681061.940 ...",68639,979882,Adams County,106.532539


In [44]:
dfpivot = pd.pivot_table(dfsjoin,index=['ID','APN'], aggfunc={'FID':len, 'areas':'mean'}).reset_index()

In [46]:
dfpolynew = adams.merge(dfpivot, how='left',on='ID')

In [48]:
dfpolynew.rename({'APN_x':'APN', 'FID': 'Sum_Within', 'areas':'mnBlgArea'}, axis='columns',inplace=True)

In [54]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [55]:
adamsParcBlds = parcelfunks.sumWithin(adams,adamsBuildings)

In [56]:
adamsParcBlds

,APN,APN2,STATE,COUNTY,FIPS,SIT_HSE_NU,SIT_DIR,SIT_STR_NA,SIT_STR_SF,SIT_FULL_S,...,MINY,MAXX,MAXY,VERSION,QUANTARIUM,ATTDATE,geomCalc,geometry,Sum_Within,mnBlgArea
0,155700000045,1,CO,Adams,08001,None,None,None,None,None,...,39.978658,-104.057487,39.985909,12/20/2016,None,20211030,630151.6230,"POLYGON ((-681231.654 306696.813, -681999.276 ...",0.0,0.000000
1,155700000007,2,CO,Adams,08001,None,None,None,None,None,...,39.996597,-104.057978,40.001045,12/20/2016,None,20211030,384020.4328,"POLYGON ((-681449.948 309232.732, -681450.962 ...",0.0,0.000000
2,155700000150,3,CO,Adams,08001,None,None,None,None,None,...,39.993095,-104.038983,39.997066,12/20/2016,None,20211030,669224.3293,"POLYGON ((-679532.942 308190.972, -680326.252 ...",0.0,0.000000
3,155700000149,4,CO,Adams,08001,None,None,None,None,None,...,39.996835,-104.039052,40.000994,12/20/2016,None,20211030,532267.7586,"POLYGON ((-680646.427 309153.695, -680646.245 ...",0.0,0.000000
4,155700000006,5,CO,Adams,08001,16704,None,MIMOSA MILE,RD,16704 MIMOSA MILE RD,...,39.996766,-104.052995,40.000949,12/20/2016,22098024,20211030,187514.4901,"POLYGON ((-680680.353 308693.941, -681061.940 ...",2.0,162.515728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118833,181333111009,166570,CO,Adams,08001,None,None,None,None,166570 2315 TROUT RD,...,39.749104,-104.330119,39.749356,12/20/2016,None,20211030,977.4080,"POLYGON ((-706475.383 283060.727, -706510.315 ...",0.0,0.000000
118834,182504302022,166571,CO,Adams,08001,None,None,None,None,166571 2125 W 64TH AVE,...,39.812962,-105.010913,39.813580,12/20/2016,None,20211030,5394.8983,"POLYGON ((-763401.996 295518.311, -763365.602 ...",0.0,0.000000
118835,182504302021,166572,CO,Adams,08001,None,None,None,None,166572 2075 W 64TH AVE,...,39.812963,-105.011479,39.815228,12/20/2016,22244250,20211030,40742.1354,"POLYGON ((-763622.608 295727.276, -763483.233 ...",0.0,0.000000
118836,182504302020,166573,CO,Adams,08001,None,None,None,None,166573 2175 W 64TH AVE,...,39.813529,-105.009931,39.815120,12/20/2016,None,20211030,21055.6189,"POLYGON ((-763366.706 295686.630, -763315.925 ...",0.0,0.000000


In [78]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [79]:
adams_dir

'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\counties\\08001'

In [80]:
parcelfunks.parcelMHPJoin2(adams_dir)

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08001\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [16]:
adamsResults = gpd.read_file(os.path.join(adams_dir,'08001multi100_less5.gpkg'),layer='MH_parcels')

In [31]:
adamsResults['dup'] = adamsResults.duplicated(subset=['geometry'],keep=False)

In [54]:
adamsResults.loc[adamsResults['dup'] == True]

,APN,APN2,Sum_Within,mnBlgArea,intLen,intZscore,extLen1,extZscore1,MHPID,MH_NAME,...,MH_CITY,MH_STATE,MH_ZIP,MH_STATUS,MH_COUNTY_FIPS,MH_LATITUDE,MH_LONGITUDE,distances,geometry,dup
28,172118000028,108665,258.0,127.128101,0,0.02231,16,2.002476,38269.0,NOT AVAILABLE,...,THORNTON,CO,80229,OPEN,08001,39.877693,-104.932114,0.0,"MULTIPOLYGON (((-755979.854 302035.048, -75599...",True
30,172118000028,108665,258.0,127.128101,0,0.02231,16,2.002476,38270.0,NOT AVAILABLE,...,THORNTON,CO,80229,OPEN,08001,39.877583,-104.936093,0.0,"MULTIPOLYGON (((-755979.854 302035.048, -75599...",True


In [61]:
bocoPath = os.path.join(co_22_dir,county_dir,'08013')
parcelfunks.parcelMHPJoin2(bocoPath)

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08013\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [63]:
boResults = gpd.read_file(os.path.join(bocoPath,'08013multi100_less5.gpkg'),layer='MH_parcels')
boResults['dup'] = boResults.duplicated(subset=['geometry'],keep=False)
boResults.loc[boResults['dup']==True]

,APN,APN2,Sum_Within,mnBlgArea,intLen,intZscore,extLen1,extZscore1,MHPID,MH_NAME,...,MH_CITY,MH_STATE,MH_ZIP,MH_STATUS,MH_COUNTY_FIPS,MH_LATITUDE,MH_LONGITUDE,distances,geometry,dup
19,157502300020,219690809,39.0,65.308106,0,0.04432,7,0.003392,38249.0,NOT AVAILABLE,...,LAFAYETTE,CO,80026,OPEN,08013,39.991468,-105.090219,0.0,"MULTIPOLYGON (((-768052.014 315988.455, -76817...",True
27,157502300020,219690809,39.0,65.308106,0,0.04432,7,0.003392,38250.0,NOT AVAILABLE,...,LAFAYETTE,CO,80026,OPEN,08013,39.991449,-105.089248,0.0,"MULTIPOLYGON (((-768052.014 315988.455, -76817...",True


In [68]:
jeffcoPath = os.path.join(co_22_dir,county_dir,'08059')
parcelfunks.parcelMHPJoin2(jeffcoPath)

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08059\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [69]:
jeffResults = gpd.read_file(os.path.join(jeffcoPath,'08059multi100_less5.gpkg'),layer='MH_parcels')

In [76]:
adamsResults.loc[adamsResults['MHPID']==24842.0]

,APN,APN2,Sum_Within,mnBlgArea,intLen,intZscore,extLen1,extZscore1,MHPID,MH_NAME,...,MH_CITY,MH_STATE,MH_ZIP,MH_STATUS,MH_COUNTY_FIPS,MH_LATITUDE,MH_LONGITUDE,distances,geometry,dup


In [77]:
adams

NameError: name 'adams' is not defined

In [89]:
def parcelMHPJoin2(pFilePath):
    fips = pFilePath.split('\\')[-1]
    mhpPath = os.path.join(pFilePath,fips+'_mhps_OG.gpkg')
    if os.path.exists(mhpPath):
        parcel = gpd.read_file(os.path.join(pFilePath,'parcels.shp'))
        parcel.to_crs(crs='ESRI:102003', inplace=True)
        parcel.drop_duplicates(subset=['geometry'], inplace=True)
        buildings = gpd.read_file(os.path.join(pFilePath,fips+'_buildings.shp'))
        buildings.to_crs(crs='ESRI:102003', inplace=True)
        parcel = parcelfunks.sumWithin(parcel,buildings)
        parcel['geometry'] = parcel['geometry'].simplify(1)
        parcel['intLen'] = parcel.apply(lambda row: parcelfunks.interiorLen(row.geometry), axis=1)
        parcel['intZscore'] = np.abs(stats.zscore(parcel['intLen']))
        parcel.drop(parcel[parcel.intLen >= 20].index, inplace=True) #dropping outlier inner geometries
        parcel.reset_index(inplace=True)
        parcel['extLen1'] = parcel.apply(lambda row: parcelfunks.exteriorLen(row.geometry), axis=1)
        parcel['extZscore1'] = np.abs(stats.zscore(parcel['extLen1']))
        parcel.drop(parcel[(parcel['extZscore1'] > 3) & (parcel['Sum_Within'] < 10)].index, inplace=True) #dropping outlier geometries
        parcel.drop(parcel[parcel['Sum_Within'] < 1].index, inplace=True)
        parcel.reset_index(inplace=True)
        columns = ['APN', 'APN2', 'intLen','intZscore', 'extLen1','extZscore1', 'Sum_Within', 'mnBlgArea', 'geometry']
        drops = [c for c in parcel.columns if c not in columns]
        parcel.drop(drops, axis=1, inplace=True)        
        mobileHomes = gpd.read_file(mhpPath)
        mobileHomes.to_crs(crs='ESRI:102003', inplace=True) 
        if parcel.crs != mobileHomes.crs:
            parcel.to_crs(mobileHomes.crs, inplace=True)
        phomes = gpd.sjoin_nearest(parcel, mobileHomes, how='inner', max_distance=100.0, distance_col='distances')
        #phomes.drop('index_right', axis=1, inplace=True)
        #phomes = phomes.sort_values(['MHPID','Sum_Within'], ascending=False).drop_duplicates(subset=['MHPID'], keep='first')
        phomes.drop(phomes[(phomes['Sum_Within'] < 10) & (phomes['mnBlgArea'] > 175)].index, inplace=True)
        phomes.drop(phomes[phomes['mnBlgArea'] > 200].index, inplace=True)
        phomes.drop(phomes[phomes['Sum_Within'] < 5].index, inplace=True) #drop matched parcels with less than 5 buildings
        #phomes.drop_duplicates(subset=['geometry'], inplace=True)
        #if len(phomes) > 0:
        #    phomes.to_file(os.path.join(pFilePath,pFilePath.split('\\')[-1]+'multi100_less5.gpkg'),driver='GPKG', layer='MH_parcels')
        #else:
        #    with open(exceptPath, 'a', newline='', encoding='utf-8') as f: #need to figure out a better way. Is it overwriting?
        #        writer = csv.writer(f)
        #        writer.writerow([pFilePath.split('\\')[-1],'NO JOIN'])
        return phomes

In [90]:
adamsResults = parcelMHPJoin2(adams_dir)

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08001\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [92]:
adamsResults['MHPID']

63380    24859.0
2087     12209.0
3112     38158.0
3045     24864.0
3224     38157.0
          ...   
61528    38240.0
61545    38238.0
61678    38236.0
61750    12087.0
61756    12084.0
Name: MHPID, Length: 62, dtype: float64

In [99]:
missedPoints = gpd.read_file(os.path.join(co_22_dir,'08_MHPs_OG.gpkg'), layer='08_mhps_non_matched_multi100_less5').to_crs(crs='ESRI:102003')

In [100]:
buffed = missedPoints.buffer(100)

In [102]:
buffed.to_file(os.path.join(co_22_dir,'08_MHPs_OG.gpkg'), layer='08_missed_buffs')

In [112]:
stateFinalDF = pd.DataFrame()

for path in parcelsPaths:
    if os.path.exists(os.path.join(path,'MHP_'+path.split('\\')[-1]+'double.csv')):
        
        countyDF = pd.read_csv(os.path.join(path,'MHP_'+path.split('\\')[-1]+'double.csv'), dtype={'STATEFP10':str,'COUNTYFP10':str,'TRACTCE10':str,'BLOCKCE10':str,'GEOID10':str,'MTFCC10':str,'UACE10':str,'GEOID10':str,'GEOID10':str, 'MH_COUNTY_FIPS':str, 'MHPID':str})
        stateFinalDF = pd.concat([stateFinalDF,countyDF])
        
stateFinalDF.drop(stateFinalDF.filter(regex='Unnamed*').columns,axis=1, inplace=True)
stateFinalDF.to_csv(r'c:/users/phwh9568/data/parcelatlas/CO_2022/Colorado_Final_double.csv')

C:\Users\phwh9568\AppData\Local\Temp\ipykernel_248\3066670642.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stateFinalDF = pd.concat([stateFinalDF,countyDF])
C:\Users\phwh9568\AppData\Local\Temp\ipykernel_248\3066670642.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stateFinalDF = pd.concat([stateFinalDF,countyDF])
C:\Users\phwh9568\AppData\Local\Temp\ipykernel_248\3066670642.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a 